In [10]:
import pandas as pd
fhlo = pd.read_csv("../../input_data/fully_hand_labelled_output.csv")

In [11]:
fhlo.Roken = fhlo.Roken.astype('category').cat.codes
fhlo.Alcohol = fhlo.Alcohol.astype('category').cat.codes
fhlo.Drugs = fhlo.Drugs.astype('category').cat.codes
fhlo["label"] = fhlo.Roken
fhlo2 = fhlo[["text", "label"]]

In [12]:
train=fhlo2.sample(frac=0.8,random_state=200)
test=fhlo2.drop(train.index)

In [13]:
train.to_csv('train.csv')
test.to_csv('test.csv')

In [23]:
train.isnull().sum(axis = 1)

4522    0
1815    0
3316    0
246     0
3831    0
       ..
3088    0
3703    0
4078    0
3058    0
499     0
Length: 3760, dtype: int64

In [15]:
train.columns

Index(['text', 'label'], dtype='object')

In [26]:
from datasets import load_dataset

file_dict = {
  "train" : "train.csv",
  "test" : "test.csv"
}

dataset = load_dataset(
  'csv',
  data_files=file_dict,
  delimiter=",",
  column_names=['text', 'label'],
  skiprows=[0, 1583]
)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /home/hjmuizelaar/.cache/huggingface/datasets/csv/default-89e66b63a23b452d/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [31]:
dataset['train'][0]

{'text': "Anamnese: \tIndicatie  Sectio caesareaAlgemene anamneseToestemming voor het opvragen van externe informatie: neeMedische verklaring patiÃ«nt neeGeen allergieÃ«n.Geen medicatie. Op dit moment zwanger: jaFamiliaire ziektes: geen.Eerder geopereerd, Keizersnede - 28/05/2014Blinde dorm verwijderen - ongeveer 1987 jaarAnesthesie: spinaal.Problemen bij eerdere operatie(s): nee.IntoxicatiesDe patiÃ«nt rookte en is gestopt met roken. De patiÃ«nt drinkt geen alcohol. De patiÃ«nt gebruikt geen drugs. Tractus CirculatoriusInspanningstolerantie voordat de aandoening optrad: ik kan stevig doorwandelen of fietsen. MET-score: 6PatiÃ«nt ervaart de volgende problemen bij het verrichten van lichamelijke inspanning: ik word alleen kortademig bij zware inspanningDyspnoe-score: 1.neeTractus Neurologicus.Tractus Overige InterneSyndroom/niet gevraagde ziekte: AambeienAdditionele anamnesePatiÃ«nt is op de hoogte van en akkoord met de aard en het doel van de voorgenomen ingreep. \t\tLichamelijk onderz

In [44]:
from datasets import ClassLabel

dataset['train'] = dataset['train'].cast_column("label", ClassLabel(num_classes=4, names=["Geen gebruiker", "Huidige gebruiker", "Niets gevonden", "Voormalige gebruiker"]))
dataset['test'] = dataset['test'].cast_column("label", ClassLabel(num_classes=4, names=["Geen gebruiker", "Huidige gebruiker", "Niets gevonden", "Voormalige gebruiker"]))

Casting the dataset:   0%|          | 0/3759 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/940 [00:00<?, ? examples/s]

In [45]:
dataset["test"][5]

{'text': "Anamnese: \t Verkort consult: Voorgeschiedenis:\tLongembolie\tCOPD Gold II\tTIA \tkrachtsverlies re been\t\t\tAnamnese: \tIndicatie  ebusAlgemene anamneseToestemming voor het opvragen van externe informatie: jaMedische verklaring patiÃ«nt neeBeroep/werkzaamheden: geen aowerGeen allergieÃ«n.Medicatie, 1x pufje + 1x bloedverdunner fraxiparine. Bloedverdunners. Familiaire ziektes: geen..IntoxicatiesDe patiÃ«nt rookte en is gestopt met roken sinds 2018. De patiÃ«nt drinkt alcohol, minder dan 2 eenheden per dag.De patiÃ«nt gebruikt geen drugs. Tractus CirculatoriusInspanningstolerantie voordat de aandoening optrad: ik doe zwaar huishoudelijk werk (bedden opmaken, meubels verschuiven) of werk flink in de tuin. MET-score: 7PatiÃ«nt ervaart de volgende problemen bij het verrichten van lichamelijke inspanning: ik word alleen kortademig bij zware inspanningDyspnoe-score: 1.neeTractus RespiratoriusLongziekten, COPD, classificatie COPD: GOLD 2 (matig).Toelichting pulmonaal: gebruikt een 

In [46]:
dataset["train"].features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['Geen gebruiker', 'Huidige gebruiker', 'Niets gevonden', 'Voormalige gebruiker'], id=None),
 '__index_level_0__': Value(dtype='int64', id=None)}

In [47]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("tokenizer")

In [48]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [49]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/3759 [00:00<?, ? examples/s]

Map:   0%|          | 0/940 [00:00<?, ? examples/s]

In [50]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("thesis-hielke/HAGALBERT", num_labels=4)

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at thesis-hielke/HAGALBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [51]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [52]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [53]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [54]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [57]:
from transformers import DataCollatorForLanguageModeling


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics,
)

In [58]:
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 